In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.pardir)))

In [ ]:
from pyoscomp.input.reader import ScenarioInputReader
import numpy as np

In [84]:
reader = ScenarioInputReader("../start/sample_data/")
reader.read_all_csvs()

In [91]:
region_df = reader.csv_data["REGION"]
regions = np.array(["DEFAULT_REGION"]) if region_df is None or region_df.empty else region_df.values.flatten()

In [98]:
import pypsa
n = pypsa.Network()
n.add("Carrier", "AC", color="lightblue")
n.add("Bus", np.array(["A", "B", "C"]))
n.set_snapshots(np.array(["TS1", "TS2", "TS3", "TS4"]))

In [105]:
n.buses.index +" load"

Index(['A load', 'B load', 'C load'], dtype='object', name='name')

In [54]:
year_df = reader.csv_data.get("YEAR")
season_df = reader.csv_data.get("SEASON")
daytype_df = reader.csv_data.get("DAYTYPE")
dailytimebracket_df = reader.csv_data.get("DAILYTIMEBRACKET")

In [ ]:
reader.years = year_df.values.flatten()
reader.seasons = np.array(["ANNUAL"]) if season_df.empty else season_df.values.flatten()
reader.daytypes = np.array(["ALLDAYS"]) if daytype_df.empty else daytype_df.values.flatten()
reader.dailytimebrackets = np.array(["ALLHOURS"]) if dailytimebracket_df.empty else dailytimebracket_df.values.flatten()

In [73]:
from itertools import product
timelabels = [f"{y}-{s}-{d}-{t}" for y, s, d, t in product(
            reader.years, reader.seasons, reader.daytypes, reader.dailytimebrackets)]

In [74]:
reader.seasons

array(['ANNUAL'], dtype='<U6')

In [75]:
timelabels

['2025-ANNUAL-ALLDAYS-ALLHOURS',
 '2026-ANNUAL-ALLDAYS-ALLHOURS',
 '2027-ANNUAL-ALLDAYS-ALLHOURS',
 '2028-ANNUAL-ALLDAYS-ALLHOURS',
 '2029-ANNUAL-ALLDAYS-ALLHOURS',
 '2030-ANNUAL-ALLDAYS-ALLHOURS',
 '2031-ANNUAL-ALLDAYS-ALLHOURS',
 '2032-ANNUAL-ALLDAYS-ALLHOURS',
 '2033-ANNUAL-ALLDAYS-ALLHOURS',
 '2034-ANNUAL-ALLDAYS-ALLHOURS',
 '2035-ANNUAL-ALLDAYS-ALLHOURS',
 '2036-ANNUAL-ALLDAYS-ALLHOURS',
 '2037-ANNUAL-ALLDAYS-ALLHOURS',
 '2038-ANNUAL-ALLDAYS-ALLHOURS',
 '2039-ANNUAL-ALLDAYS-ALLHOURS',
 '2040-ANNUAL-ALLDAYS-ALLHOURS',
 '2041-ANNUAL-ALLDAYS-ALLHOURS',
 '2042-ANNUAL-ALLDAYS-ALLHOURS',
 '2043-ANNUAL-ALLDAYS-ALLHOURS',
 '2044-ANNUAL-ALLDAYS-ALLHOURS',
 '2045-ANNUAL-ALLDAYS-ALLHOURS',
 '2046-ANNUAL-ALLDAYS-ALLHOURS',
 '2047-ANNUAL-ALLDAYS-ALLHOURS',
 '2048-ANNUAL-ALLDAYS-ALLHOURS',
 '2049-ANNUAL-ALLDAYS-ALLHOURS',
 '2050-ANNUAL-ALLDAYS-ALLHOURS']